In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import re

# Rutas donde están almacenados los datos
path_1MJ_YESfeel_lower = "/home/matias/Simulations/mi_fargo3d/outputs/flyby2d_1MJ_YESfeel_lower/"
path_1MJ_NOfeel_lower = "/home/matias/Simulations/mi_fargo3d/outputs/flyby2d_1MJ_NOfeel_lower/"

path = path_1MJ_YESfeel_lower

# Leer variables.par
variables_par = np.genfromtxt(path + "/variables.par", dtype={'names': ("parametros", "valores"), 'formats': ("|S30", "|S300")}).tolist()
parametros_par, valores_par = [], []
for posicion in variables_par:
    parametros_par.append(posicion[0].decode("utf-8"))
    valores_par.append(posicion[1].decode("utf-8"))

def P(parametro):
    return valores_par[parametros_par.index(parametro)] 

# Parámetros de la simulación
Ninter = int(P("NINTERM"))
DT = float(P("DT"))
NX = int(P("NX"))
NY = int(P("NY"))

# Obtener Rmax
try:
    Rmax = float(P("RMAX"))
except:
    domain_y = np.genfromtxt(path + "domain_y.dat")[3:-3]
    Rmax = domain_y[-1]

def obtener_momento_angular_y_distancia(path):
    """
    Calcula el momento angular específico y la distancia radial del planeta.
    """
    planet_file = path + "planet0.dat"
    planet_data = np.genfromtxt(planet_file)
    
    x_coords = planet_data[:, 1]
    y_coords = planet_data[:, 2]
    v_azimutal = planet_data[:, 4]
    
    r_distances = np.sqrt(x_coords**2 + y_coords**2)
    momento_angular_especifico_planeta = r_distances * v_azimutal
    
    return momento_angular_especifico_planeta, r_distances

# Obtener los datos
momento_angular, r_distances = obtener_momento_angular_y_distancia(path)

# Crear array de snapshots
snapshots = np.arange(len(momento_angular))

# Identificar segmentos donde el planeta está dentro y fuera del disco
inside_disk = r_distances < Rmax

# Función modificada para crear segmentos y retornar la condición
def crear_segmentos(x, y, condition):
    segments_x = []
    segments_y = []
    segments_condition = []
    current_segment_x = []
    current_segment_y = []
    current_condition = condition[0]  # Iniciar con la primera condición

    for i in range(len(x)):
        current_segment_x.append(x[i])
        current_segment_y.append(y[i])

        # Si la condición cambia o es el último punto, cerramos el segmento
        if i < len(x) - 1:
            if condition[i] != condition[i+1]:
                segments_x.append(current_segment_x)
                segments_y.append(current_segment_y)
                segments_condition.append(current_condition)
                current_segment_x = []
                current_segment_y = []
                current_condition = condition[i+1]
        else:
            # Añadir el último segmento
            segments_x.append(current_segment_x)
            segments_y.append(current_segment_y)
            segments_condition.append(current_condition)

    return segments_x, segments_y, segments_condition

# Crear segmentos para los datos
segments_x, segments_y, segments_condition = crear_segmentos(snapshots, momento_angular, inside_disk)

# Graficar los segmentos con colores diferentes
plt.figure()

colors = {True: 'red', False: 'blue'}
labels_done = {True: False, False: False}

for x_seg, y_seg, cond in zip(segments_x, segments_y, segments_condition):
    if not labels_done[cond]:
        plt.plot(x_seg, y_seg, color=colors[cond], label='Dentro del disco' if cond else 'Fuera del disco')
        labels_done[cond] = True
    else:
        plt.plot(x_seg, y_seg, color=colors[cond])

plt.xlabel("Número de Snapshot")
plt.ylabel("Momento Angular Específico [UA² / año]")
plt.title("Evolución del Momento Angular Específico del Planeta")
plt.grid(True)
plt.legend()
plt.show()
